### libraries

In [20]:
# Import pandas & yfinance
import pandas as pd
import yfinance as yf
# Import numpy
import numpy as np
from numpy.linalg import multi_dot
# Set numpy random seed
np.random.seed(23)
# Import cufflinks
import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((1000,600)))
# Import plotly express for EF plot
import plotly.express as px
px.defaults.width, px.defaults.height = 1000, 600
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)

## Retrieve data

In [21]:
##We will retrieve price data from our list of stocks as before to build our portfolio

# Specify assets / stocks
# international etf portfolio :['SPY', 'GLD', 'IWM', 'VWO', 'BND']
# indian stocks : bank, consumer goods, diversified, it, consumer durables
# ['HDFCBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT']
assets = ['SPY', 'GLD', 'IWM', 'VWO', 'BND']
assets.sort()
# Number of assets
numofasset = len(assets)
# Number of portfolio for optimization
numofportfolio = 10000


In [22]:
# Get yahoo tickers for indian stocks
#yahooticker = [x+'.NS' for x in assets]
# Fetch / read data for multiple stocks at once
df = yf.download(assets, start='2015-01-01', end='2022-12-31', progress=False)['Adj Close']
df.columns = assets
# write data to file for future use
# df.to_csv('data/india_stocks.csv')
# Read from file
#df = pd.read_csv('data/india_stocks.csv', index_col=0, parse_dates=True)
# Display dataframe
df

,BND,GLD,IWM,SPY,VWO
Date,,,,,
2015-01-02,64.7270,114.08,105.3527,174.2654,30.3915
2015-01-05,64.9150,115.80,103.9442,171.1182,29.9075
2015-01-06,65.1030,117.12,102.1460,169.5064,29.7616
2015-01-07,65.1421,116.43,103.4039,171.6187,30.4453
2015-01-08,65.0404,115.94,105.1578,174.6640,30.9600
...,...,...,...,...,...
2022-12-23,69.8835,167.26,171.8297,376.0091,37.3613
2022-12-27,69.3439,168.67,170.6768,374.5263,37.9495
2022-12-28,69.2476,167.91,168.0263,369.8718,37.3999


## Visualize time series

In [23]:
# Plot price history
df['2022':].normalize().iplot(kind='line')

In [24]:
# Dataframe of returns and volatility
returns = df.pct_change().dropna()
annual_returns = round(returns.mean()*260*100,2)
annual_stdev = round(returns.std()*np.sqrt(260)*100,2)
df1 = pd.DataFrame({
'Ann Ret': annual_returns,
'Ann Vol': annual_stdev
})


In [25]:
df1

,Ann Ret,Ann Vol
BND,1.00,5.25
GLD,6.15,14.32
IWM,9.07,23.37
SPY,11.72,18.92
VWO,5.02,21.27


In [26]:
# Plot annualized return and volatility
df1.iplot(
kind='bar',
shared_xaxes=True,
orientation='h'
)


## Portfolio composition

In [27]:
df1.reset_index().iplot(
kind='pie',
labels='index',
values='Ann Ret',
textinfo='percent+label',
hole=0.4
)


## Portfolio simulation

In [28]:
def portfolio_simulation(returns):
# Initialize the lists
    rets = []; vols = []; wts = []
    # Simulate 5,000 portfolios
    for i in range (numofportfolio):
    # Generate random weights
        weights = np.random.random(numofasset)
        # Set weights such that sum of weights equals 1
        weights /= np.sum(weights)
        # Portfolio statistics
        rets.append(weights.T @ np.array(returns.mean() * 260))
        vols.append(np.sqrt(multi_dot([weights.T, returns.cov()*260, weights])))
        wts.append(weights)
    # Create a dataframe for analysis
    data = {'port_rets': rets, 'port_vols': vols}
    for counter, symbol in enumerate(returns.columns.tolist ()) :
        data[symbol+' weight'] = [w[counter] for w in wts]
    portdf = pd.DataFrame(data)
    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']
    return round(portdf,4)


In [30]:
## min sharpe ratio

# Create a dataframe for analysis
temp = portfolio_simulation(returns)
temp.head()
# Get the max sharpe portfolio stats
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets       0.0843
port_vols       0.1151
BND weight      0.0579
GLD weight      0.4580
IWM weight      0.0070
SPY weight      0.4619
VWO weight      0.0152
sharpe_ratio    0.7318
Name: 9002, dtype: float64

In [31]:
 # Verify the above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,10000.0,0.0660,0.0103,0.0243,0.0592,0.0659,0.0728,0.1021
port_vols,10000.0,0.1285,0.0220,0.0594,0.1132,0.1271,0.1426,0.2200
BND weight,10000.0,0.2007,0.1131,0.0000,0.1130,0.2027,0.2777,0.7795
GLD weight,10000.0,0.1988,0.1133,0.0000,0.1092,0.1987,0.2766,0.6743
IWM weight,10000.0,0.2012,0.1148,0.0001,0.1106,0.2002,0.2778,0.9124
SPY weight,10000.0,0.2011,0.1131,0.0000,0.1136,0.2004,0.2773,0.7585
VWO weight,10000.0,0.1982,0.1147,0.0000,0.1051,0.1964,0.2778,0.7187
sharpe_ratio,10000.0,0.5191,0.0666,0.2888,0.4741,0.5172,0.5628,0.7318


## Visualize simulated portfolio

In [32]:
# Plot simulated portfolio
fig = px.scatter(
    temp, x='port_vols', y='port_rets', color='sharpe_ratio',
    labels={'port_vols': 'Expected Volatility', 'port_rets': 'Expected Return','sharpe_ratio': 'Sharpe Ratio'},
    title="Monte Carlo Simulated Portfolio"
    ).update_traces(mode='markers', marker=dict(symbol='cross'))

# Plot max sharpe
fig.add_scatter(
    mode='markers',
    x=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']],
    y=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']],
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name = 'Max Sharpe'
).update(layout_showlegend=False)


# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()


## Constrained optimization

In [33]:
# Import optimization module from scipy
# sco.minimize?
import scipy.optimize as sco

In [34]:
# Portfolio statistics
def portfolio_stats(weights):
    weights = np.array(weights)
    port_rets = weights.T @ np.array(returns.mean() * 260)
    port_vols = np.sqrt(multi_dot([weights.T, returns.cov() * 260, weights]))

    return np.array([port_rets, port_vols, port_rets/port_vols])


# Minimize the volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]

# Minimize the variance
def min_variance(weights):
    return portfolio_stats(weights)[1]**2

# Maximizing sharpe ratio
def max_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]

In [35]:
# Specify constraints, bounds and initial weights
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0,1) for x in range(numofasset))

initial_wts = numofasset*[1./numofasset]


In [36]:
# Optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(max_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
# Optimizing for minimum variance
opt_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)


In [37]:
opt_sharpe

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.7430319810957837
       x: [ 0.000e+00  4.723e-01  1.576e-16  5.277e-01  1.040e-16]
     nit: 8
     jac: [ 6.464e-03  9.640e-04  3.061e-01 -8.653e-04  4.825e-01]
    nfev: 49
    njev: 8

In [38]:
opt_var

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.00267457847649223
       x: [ 9.499e-01  7.103e-03  7.256e-03  2.070e-02  1.500e-02]
     nit: 14
     jac: [ 5.361e-03  5.453e-03  5.290e-03  4.923e-03  5.147e-03]
    nfev: 84
    njev: 14

In [39]:
# Efficient Frontier
targetrets = np.linspace(0.01,0.1065,100)
tvols = []
for tr in targetrets:
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr},
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=ef_cons)
    tvols.append(opt_ef['fun'])
targetvols = np.array(tvols)

In [40]:
# Create EF Dataframe for plotting
efport = pd.DataFrame({
'targetrets' : np.around(100*targetrets,2),
'targetvols': np.around(100*targetvols,2),
'targetsharpe': np.around(targetrets/targetvols,2)
    })
efport.head()

,targetrets,targetvols,targetsharpe
0,1.00,5.25,0.19
1,1.10,5.20,0.21
2,1.19,5.18,0.23
3,1.29,5.17,0.25
4,1.39,5.17,0.27


In [41]:
# Plot efficient frontier portfolio
fig = px.scatter(
    efport, x='targetvols', y='targetrets', color='targetsharpe',
    labels={'targetrets': 'Expected Return', 'targetvols': 'Expected,Volatility','targetsharpe': 'Sharpe Ratio'},
    title="Efficient Frontier Portfolio"
    ).update_traces(mode='markers', marker=dict(symbol='cross'))

# Plot maximum sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_sharpe['x'])[1]],
    y=[100*portfolio_stats(opt_sharpe['x'])[0]],
    marker=dict(color='red', size=20, symbol='star'),
    name = 'Max Sharpe'
    ).update(layout_showlegend=False)

# Plot minimum variance portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]],
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='green', size=20, symbol='star'),
    name = 'Min Variance'
    ).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()
